In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [7]:
df = pd.read_parquet('data/yellow_tripdata_2023-01.parquet')

In [8]:
df.shape

(3066766, 19)

Q1: How many columns are there?<br>
ِAnswer: 19

In [9]:
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [10]:
df['duration'].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

Q2: What's the standard deviation of the trips duration in January?<br>
Answer: 42.59

In [11]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [14]:
df.shape

(3009173, 20)

In [16]:
Q3 = (3009173/3066766) * 100
Q3

98.1220282212598

Q3: What fraction of the records left after you dropped the outliers?<br>
Answer: 98%

In [19]:
def read_datafram(filname):
    df = pd.read_parquet(filname)

    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    features = ['PULocationID', 'DOLocationID']
    
    df[features ] = df[features ].astype(str)
    
    return df

In [20]:
train_df = read_datafram('data/yellow_tripdata_2023-01.parquet')
val_df = read_datafram('data/yellow_tripdata_2023-02.parquet')

In [21]:
features = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = train_df[features].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
y_train = train_df['duration'].values

val_dicts = val_df[features].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = val_df['duration'].values

In [23]:
X_train.shape

(3009173, 515)

Q4: What's the dimensionality of this matrix (number of columns)?<br>
Answer: 515

In [27]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

rmse_train  = mean_squared_error(y_train, y_pred, squared=False)

print('RMSE on train: ', rmse_train)

RMSE on train:  7.649261930819891


Q5: What's the RMSE on train?<br>
Answer: 7.65 or the closest number to it

In [28]:
y_pred = lr.predict(X_val)

rmse_val = mean_squared_error(y_val, y_pred, squared=False)

print('RMSE on validation: ', rmse_val)

RMSE on validation:  7.811817675774269


Q6: What's the RMSE on validation?<br>
Answer 7.81  or the closest number to it